In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.transforms.v2 import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
from torchvision.models import resnet50
from sklearn.metrics import classification_report

In [14]:
device = 'cuda'

# Этап 1. Загрузка и предобработка данных

In [7]:
train_path = 'data/ogyeiv2/train'
test_path = 'data/ogyeiv2/test'

In [8]:
train_dataset = ImageFolder(train_path)
test_dataset = ImageFolder(test_path)

In [9]:
class TransformDataset(Dataset):
  def __init__(self, dataset, transforms):
    super(TransformDataset, self).__init__()
    self.dataset = dataset
    self.transforms = transforms

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    x, y = self.dataset[idx]
    return self.transforms(x), y

In [ ]:
train_transforms = Compose([
    RandomHorizontalFlip(p=0.2),
    RandomVerticalFlip(p=0.2),
    RandomRotation([-5, 5], fill=255.),
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.5), (0.5))
])

test_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.5), (0.5))
])

In [11]:
train = TransformDataset(train_dataset, train_transforms)
test= TransformDataset(test_dataset, test_transforms)

In [12]:
batch_size = 32
train_loader = DataLoader(train,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(test,
                        batch_size=batch_size,
                        shuffle=False)

In [9]:
print('Размер train-датасета: ', len(train_dataset))
print('Размер test-датасета: ', len(test_dataset))
print('Кол-во классов: ', len(train_dataset.classes))

Размер train-датасета:  2352
Размер test-датасета:  504
Кол-во классов:  84


# Этап 2. Объявление модели

In [ ]:
model = resnet50(weights='IMAGENET1K_V2')

In [ ]:
# Меняем последний слой нейросети для предсказания 84 классов
model.fc = nn.Linear(in_features=2048, out_features=84, bias=True)

In [15]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Этап 3. Обучение или дообучение

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) 

In [30]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    for batch_index, data in enumerate(train_loader):
        # Извлечение батча
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # Обнуление градиентов
        optimizer.zero_grad()
        # Прямое распространение
        outputs = model(inputs)
        # Подсчёт ошибки
        loss = criterion(outputs, labels)
        # Обратное распространение
        loss.backward()
        # Обновление весов
        optimizer.step()

        # Суммирование ошибки за последние 100 батчей
        running_loss += loss.item()
        if batch_index == 73:
            last_loss = running_loss / 73. # средняя ошибка за 100 батчей
            print(f'Эпоха: {epoch_index}, батч: {batch_index}, ошибка {last_loss}')
            running_loss = 0.

    return last_loss

In [ ]:
EPOCHS = 15

best_vloss = 1e5

for epoch in range(EPOCHS):
    print(f'Эпоха {epoch}')

    # Перевод модели в режим обучения
    model.train(True)
    # Эпоха обучения
    avg_loss = train_one_epoch(epoch)

    # Перевод модели в режим валидации
    model.eval()
    running_vloss = 0.0

    # Валидация
    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)
            voutputs = model(vinputs)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)

    # Сохранение лучшей модели
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = f'meds_classifier_{epoch}.pt'
        torch.save(model.state_dict(), model_path)

    print(f'В конце эпохи ошибка train {avg_loss}, ошибка val {avg_vloss}')

Эпоха 0
Эпоха: 0, батч: 73, ошибка 0.09332129493840549
В конце эпохи ошибка train 0.09332129493840549, ошибка val 29.38132667541504
Эпоха 1
Эпоха: 1, батч: 73, ошибка 0.06334399998392144
В конце эпохи ошибка train 0.06334399998392144, ошибка val 1.319373369216919
Эпоха 2
Эпоха: 2, батч: 73, ошибка 0.09897583220409203
В конце эпохи ошибка train 0.09897583220409203, ошибка val 17.412233352661133
Эпоха 3
Эпоха: 3, батч: 73, ошибка 0.05775509235666019
В конце эпохи ошибка train 0.05775509235666019, ошибка val 24.429367065429688
Эпоха 4
Эпоха: 4, батч: 73, ошибка 0.08074471872134058
В конце эпохи ошибка train 0.08074471872134058, ошибка val 48.580909729003906
Эпоха 5
Эпоха: 5, батч: 73, ошибка 0.04489767932562693
В конце эпохи ошибка train 0.04489767932562693, ошибка val 1.3325735330581665
Эпоха 6
Эпоха: 6, батч: 73, ошибка 0.07298398970734736
В конце эпохи ошибка train 0.07298398970734736, ошибка val 0.242804616689682
Эпоха 7
Эпоха: 7, батч: 73, ошибка 0.06588137804288162
В конце эпохи оши

# Этап 4. Оценка качества

In [4]:
model = resnet50()
model.fc = nn.Linear(in_features=2048, out_features=84, bias=True)

In [15]:
model.load_state_dict(torch.load('meds_classifier_14.pt'))
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
labels_predicted = []
labels_true = []

model.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1) # argmax по всем примерам
        labels_predicted.extend(predicted.cpu().numpy())
        labels_true.extend(labels.cpu().numpy())

print(classification_report(labels_true, labels_predicted, target_names=test_dataset.classes))

                                  precision    recall  f1-score   support

                 acc_long_600_mg       0.86      1.00      0.92         6
               advil_ultra_forte       1.00      1.00      1.00         6
                   akineton_2_mg       1.00      1.00      1.00         6
      algoflex_forte_dolo_400_mg       1.00      1.00      1.00         6
           algoflex_rapid_400_mg       1.00      1.00      1.00         6
                algopyrin_500_mg       1.00      1.00      1.00         6
             ambroxol_egis_30_mg       1.00      1.00      1.00         6
                  apranax_550_mg       1.00      1.00      1.00         6
            aspirin_ultra_500_mg       1.00      1.00      1.00         6
                    atoris_20_mg       1.00      1.00      1.00         6
         atorvastatin_teva_20_mg       1.00      1.00      1.00         6
                   betaloc_50_mg       1.00      1.00      1.00         6
                        bila_git     

# Итоги

* На каких 5 классах модель ошибается чаще всего?

    Чаще всего модель ошибается на классах kalcium_magnezium_cink, tritace_5_mg, merckformin_xr_1000_mg, diclopram_75-mg_20-mg, jutavit_cink, sicor_10_mg - у этих классов самое низкое значение F1-score.

* Почему модель может ошибаться на этих классах?

    Модель может ошибаться на этих классах из-за схожести внешнего вида некоторых классов таблеток, например:
    kalcium_magnezium_cink схожи с merckformin_xr_1000_mg
    sicor_10_mg схожи с tritace_5_mg
    narva_sr_1_5_mg_retard схожи с jutavit_cink
    jutavit_c_vitamin схожи с diclopram_75-mg_20-mg.

* На каких классах модель не совершает ошибок?

    Модель не совершает ошибок на большинстве классов датасета (68 безошибочных классов из 84).

* Почему эти классы модель распознаёт безошибочно?

    Эти классы имеют больше отличительных черт (цвет, форма).

* Как можно улучшить точность классификатора?

    Для улучшения точности можно попробовать другие аугментации, либо не использовать их совсем, так как изображения в обучающем датасете практически не отличаются от изображений в валидационном (одна таблетка на нейтральном фоне, без посторонних предметов и другого шума).

* Как ещё можно проанализировать результаты и ошибки модели?

    Можно изучить изображения, полученные в результате аугментации и проанализировать, с какими именно классами модель путает классы с низким F1-score.